In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# read in csv file
file = 'full_scrape_20yr'
meta_df = pd.read_csv(file)
# display columns for sql tables
meta_df.columns


Index(['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score',
       'pickle_name', 'album_img', 'crit_rev_num', 'user_rev_num',
       'record_label', 'album_genre'],
      dtype='object')

In [3]:
# create dataframe without pickle name column
meta_sql_df = meta_df.drop(columns=['pickle_name'])
meta_sql_df.head()

,artist,album,date,week_num,meta_score,user_score,album_img,crit_rev_num,user_rev_num,record_label,album_genre
0,The Streets,Original Pirate Material,"October 22, 2002",43,90,87,https://static.metacritic.com/images/products/...,25.0,87,Locked On / Vice,Electronic
1,Tom Waits,Alice,"May 7, 2002",19,90,85,https://static.metacritic.com/images/products/...,20.0,86,Epitaph / Anti-,Rock
2,Queens of the Stone Age,Songs For The Deaf,"August 27, 2002",35,89,89,https://static.metacritic.com/images/products/...,23.0,89,Interscope,Rock
3,Spoon,Kill The Moonlight,"August 20, 2002",34,88,88,https://static.metacritic.com/images/products/...,21.0,88,Merge,Indie
4,Wilco,Yankee Hotel Foxtrot,"April 23, 2002",17,87,89,https://static.metacritic.com/images/products/...,26.0,89,Nonesuch,Rock


In [4]:
# create a column for just the year an album was created.
meta_sql_df['album_year']=pd.to_datetime(meta_sql_df['date'])
meta_sql_df['album_year'] = meta_sql_df['album_year'].dt.to_period('Y')
meta_sql_df['album_year'] = meta_sql_df.astype({'album_year': 'int64'}).dtypes

In [5]:
meta_sql_df.dtypes


artist           object
album            object
date             object
week_num          int64
meta_score        int64
user_score        int64
album_img        object
crit_rev_num    float64
user_rev_num      int64
record_label     object
album_genre      object
album_year       object
dtype: object

In [8]:
from meta_db import create_meta_db
import sqlite3

In [7]:
create_meta_db()

table 1 created
table 2 created
table 3 created


In [11]:
con = sqlite3.connect("meta_music.db")

In [10]:
meta_sql_df.to_sql("albums", con, if_exists="replace")

con.close()



In [18]:
try:
    with con:
        cur = con.cursor()
        con.execute("select * from albums where album_year=:year", {"year": "2015"})
        print(cur.fetchall())

except sqlite3.IntegrityError:
    print("couldn't add Python twice")

[]


In [ ]:
con.execute("create table lang (id integer primary key, name varchar unique)")

# Successful, con.commit() is called automatically afterwards
with con:
    con.execute("insert into lang(name) values (?)", ("Python",))

# con.rollback() is called after the with block finishes with an exception, the
# exception is still raised and must be caught
try:
    with con:
        con.execute("insert into lang(name) values (?)", ("Python",))
except sqlite3.IntegrityError:
    print("couldn't add Python twice")

# Connection object used as context manager only commits or rollbacks transactions,
# so the connection object should be closed manually
con.close()

cur = con.cursor()
cur.execute("create table lang (name, first_appeared)")

# This is the qmark style:
cur.execute("insert into lang values (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cur.executemany("insert into lang values (?, ?)", lang_list)

# And this is the named style:
cur.execute("select * from lang where first_appeared=:year", {"year": 1972})
print(cur.fetchall())

con.close()